# GDSC Training Session - EDA and Basic Modelling

<a id='Introduction'></a>
# Introduction

The first step in any data science project is to get a good understanding of the data you are dealing with. To do that you are asking questions regarding your data to gain insights in its structure, type and particularities. The questions you ask depend on the kind of data that is available but also on your previous findings. This iterative process is called exploratory data analysis (EDA). As you get more into the data science and AI realm, you will find that there are many different flavours of EDA, depending on the use case, problem setting and types of data. 

It is very common in projects for the EDA to take a lot of time. Things have changed a bit since deep learning, hyperparameter based models and non parametric feature reduction techniques have taken hold, but it is very common for the EDA to take almost as much time as the modeling. The reason why is because you are gathering a great deal of information in the EDA. What you learn impacts your further analysis but also the steps you will take for data preparation and then modelling. It also helps you gather ideas on how to best tackle the problem at hand or to verify that the task cannot be solved with the data you have available. In any case, the better you understand your problem and the associated data the more likely your are to succeed. If you are not getting closer to this you need to ask different questions about your data.

A general framework as well as some guidelines to follow for the workflow on data science projects is [CRISP-DM](https://www.sv-europe.com/crisp-dm-methodology/) or [Guerilla Analytis](https://guerrilla-analytics.net/the-principles/). The EDA stretches across several of those steps and pays into Business Understanding, Data Understanding and to some part also Data Preparation. In the context of this tutorial, we'll show you the first steps of each data science workflow step and give you ideas how to proceed further.

Now, lets get started and ask some questions regarding our data!!

*Notes:*
- *If this is your first Jupyter Notebook you might want to read [a general introduction](https://realpython.com/jupyter-notebook-introduction/) first.*
- *Do not be afraid to ask (and answer) questions! Check out the [teams channel](https://teams.microsoft.com/l/team/19%3a4017a2e9af4942e7aa157d6ec9d751b4%40thread.skype/conversations?groupId=7d77d672-dff1-4c9f-ac55-3c837c1bebf9&tenantId=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61).*

## Table of contents:
[1. Introduction](#Introduction) <br>
[2. Recap - Task of the Challenge](#Recap) <br>
[3. Exploratory Data Analysis](#EDA) <br>
&emsp; [3.1. Environment Setup](#Environment) <br>
&emsp; [3.2. Overall Data Assessment](#OverallDA) <br>
&emsp; [3.3. Data Visualization](#DataViz) <br>
[4. Modelling](#Modelling) <br>
&emsp; [4.1. Introduction](#ModelIntro) <br>
&emsp; [4.2. Training](#Training) <br>
&emsp; [4.3. Inference](#Inference) <br>
&emsp; [4.4. Submitting Results](#Submission) <br>
&emsp; [4.5. Human Validity Check](#Validation) <br>
[5. Wrap Up](#Epilogue) <br>


***

<a id='Recap'></a>
# Recap - Task of the Challenge

> The goal is to detect events or anomalies from the LoVe observatory. Your task is to build an AI that uses the different sensor inputs and detects anomalies in the overall pattern.

This is intentionally very broad and includes all kinds of anomalies. It is not dependend on which time scale they occur or in which sensor you find them. Also, there is no need to identify which kind of anomaly you see. The simple point of just finding one (no matter its kind) is enough.

## Current state of the data

The task of Global Data Science Challenge is not an easy one, especially in its bare form as the reasearcher from the LoVe Observatory have it. In order to make the problem setting better suited for the challenge we performed some preprocessing on the data and also reduced the scope of the data to only about two years from the early times of the data collection instead of the full data set that is collected to this day and beyond.

Here a quick outline of what happend between the data provided in the *raw/* folder and the *tabular/* folder:
* ADCP
    * Extraction of current speed and direction from the .nc files using [NetCDF4](https://pypi.org/project/netCDF4/) and [xarray](http://xarray.pydata.org/en/stable/)
    * Linear interpolation of the data to provide one data point per minute sample rate - before roughly one every ten minutes
    * Store data in CSV tables with one file per day of data and one data point per minute - speed and direction in one table, check the column names
    
* EK60 - Echosounder
    * Excration of the volume backscatter strengh info from the .raw files using [PyEchoLab](https://github.com/CI-CMG/pyEcholab)
    * Aggregation of the data using a mean to one data point per minute - before roughly one data point every 4 seconds
    * Cutoff of values further away from the sensor than 300 meters, so data above the ocean surface
    * Store data in CSV tables with one file per day of data and one data point per minute
* Hydrophone
    * Downsampling the .wav audio files to 12kHz maximum frequency using [librosa](https://librosa.org/doc/latest/index.html) - our research partners assured us that the information loss from this is negligable (Note: this is the state we provided to you in the *raw/* folder since the pure .wav files have an even bigger volume)
    * Performing [Long Term Spectral Average](https://libraries.io/github/tryan/LTSA) on the data to have it available in the frequeny vs. time domain
    * Store data in CSV tables with one file per day of data and one data point per minute
* Point Sensors - Temperature, Salinity, Depth, ....
    * Raw data already provided in CSV format with roughly hourly measurements
    * Linear interpolation of the data to provide one data point per minute sample rate - before roughly one every hour
    * Store data in CSV tables with one file per day of data and one data point per minute
    
These steps were performed to address mainly three issues:
* reducing the complexity of involved data types and therefore necessary libraries to use for data extraction
* reducing the data volume to something managable - we went down from around 100TB to about 500GB for raw and 50GB for tabular
* minimizing the remaining preprocessing efforts when it comes to sample rate alignment to combine the data of the different sensors

***

<a id='EDA'></a>
# Exploratory Data Analysis

<a id='Environment'></a>
## Environment Set Up

In case you missed it, please have a look at the tutorial from last week on AWS ([recording link](https://web.microsoftstream.com/video/56f52021-1c1b-455a-ae6f-9008eab77760)). Otherwise, you will not mitigate your risk that you will choke your machine with processing. Neither will you mitigate the substantial risk that you will create unnecessary costs by running unnecessarily large machines for simple tasks. Finally, be sure to know AWS S3 and Sagemaker ([Introduction video series](https://www.youtube.com/watch?v=uQc8Itd4UTs&list=PLhr1KZpdzukcOr_6j_zmSrvYnLUtgqsZz) and [Introduction Notebook](https://github.com/awslabs/amazon-sagemaker-workshop/tree/master/Introduction)), and how they work together before you undertake this challenge.

As a general rule of thumb: use small instances of the t2, t3 series for your Sagemaker Notebook and Sagemaker Studio work and only use bigger (and more expensive) instances like the c4, c5 or p2 series for training and processing jobs. An overview of the SageMaker instances can be found on the [pricing overview](https://aws.amazon.com/sagemaker/pricing/?nc1=h_ls). Also keep in mind our policies on availablle instance types from the [FAQ](http://de-mucingode1.corp.capgemini.com/gitlab/SophieY/global_data_science_challenge_4_public/-/wikis/Questions-about-AWS-account#which-instance-types-can-we-use).

In [ ]:
from pathlib import Path                                 # For handling file path operations independent of the underlying OS
from datetime import datetime, timedelta                 # For datetime handling

import numpy as np                                       # For performing matrix operations and numerical processing.
import pandas as pd                                      # For manipulating tabular data
import matplotlib.pyplot as plt                          # For plotting
import seaborn as sns                                    # For plotting

import sagemaker                                         # Interacting with AWS SageMaker
import boto3                                             # Interacting with AWS Services
from sagemaker.amazon.amazon_estimator import RecordSet  # To create a data set from protobuf format
from botocore.exceptions import ClientError              # Class for error handling

In [ ]:
role = sagemaker.get_execution_role() # Retrieve the Sagemaker Execution role, we will need to provide it to the estimator for model training later on
bucket = 'gdsc4-eu' # change this according to the bucket name assigned to your region

<a id='OverallDA'></a>
## Overall Data Assessment
The researchers have described the data for us in human readable terms with charts to illustrate their meaning.  [They can be found here](https://gdsc.ce.capgemini.com/data/) on the GDSC website.

In terms of data access there are two ways to gain general overview on how our data is stored in S3. We can leverage 
* the S3 console and have a look in the browser by using the respective link - for EU https://s3.console.aws.amazon.com/s3/buckets/gdsc4-eu
* the AWS CLI to programmatically check the "folder" contents

Also, we have the data available in two (very) different states. In S3 it is stored in "raw" as well as "tabular" form.

### Raw Data
We have already processed this data for you into csv files (tabular form), so we discourage you from going straight to the raw data at this stage (and possibly not ever).  We are listing it here for your information.


| measure_name | source_URL | data_type | unprocessed_file_type | data_payload_description | note | report_name | time_stamp | temperature | measure |
|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| Conductivity | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1044 | text | .txt | small. local processing possible | Conductivity measures water's ability to conduct electricity due to the presence or absence of certain ions and the electrical conductivity of seawater depends on temperature, salinity, and to a much smaller degree pressure (depth).  Available day wise as .txt        files Files contain conductivity – temperature pairs  | str | date | float | float |
| EK60 | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1039 | movie, raw movie | .idx, .raw | massive, local not possible | High-precision scientific echo sounder for underwater.  can operate a large number of frequencies simultaneously ranging from 10 to 500 kHz  |
| Depthsensor | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1045 | text | .txt | small. local processing possible | Used to record data such as the depth of the ocean and temperature differences. Available day wise as .txt files. Files contain depth – temperature pairs | str | date | float | float |
| ADCP | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1046 | doppler acoustic audio | .cpr, cpa | medium volume, format not accessible | Acoustic Doppler Current Profiler Used to measure how fast water is moving across an entire water column |
| hydrophone | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1048 | wave audio | .wav | massive | passive listening devices, that have one or several sensing elements sound propagates efficiently in the ocean and can often be heard many kilometres from their source sound reflects and refracts in complicated ways as it travels in the stratified ocean |
| Turbidity | https://love.statoil.com/Sensor/PreviewFiles?sensorid=1049 | text | .txt | small. local processing possible | measure of relative clarity of the water It is a measurement of the amount of light that is scattered by material in the water Highly turbid ocean waters are those with many scattering particulates in them  | str | date | float |
| chlorophyl | https://love.statoil.com/Sensor/PreviewFiles?sensorId=1050 | text | .txt | small. local processing possible | responsible for the absorption of light to provide energy for photosynthesis indicator for the amount of photosynthetic plankton, or phytoplankton, present in the ocean chlorophyll concentrations are highest where there are lots of nutrients Phytoplankton populations are influenced by climatic factors such as sea surface temperatures and winds  | str | date | float |

### Tabular Data
We will continue working with the tabular data. The first steps can also be used for checking the raw data on an overview level, but once the go more granular and ask more detailed questions about our data the methods described here will fail for the raw data.

Above we mentioned two ways of browsing the data. From here on we will go with the [AWS CLI](https://awscli.amazonaws.com/v2/documentation/api/latest/reference/s3/index.html) in order to get familiar with the prefixes (i.e. folder names) for later programmatic use. The CLI for S3 works very similar to what you are used to from unix based systems when performing operations on the file system.

In [ ]:
!aws s3 ls # list all available S3 buckets

In [ ]:
!aws s3 ls s3://gdsc4-eu/ # check the content of the bucket

In [ ]:
!aws s3 ls s3://gdsc4-eu/data/ # check the content of folder /data

Here also an overview on the folder structure we established within the S3 bucket(s):

```
<bucket_name>/
    data/
        adcp/
            raw/
                ...
            tabular/
                <date>.csv             # actual data file in tabular format
                meta_info_<date>.txt   # file containing meta info about the data from that <date>
                record_IO_<date>.pbr   # protobuf data for faster model training
        ek60/
            raw/
                ...
            tabular/
                <date>.csv             # actual data file in tabular format
                meta_info_<date>.txt   # file containing meta info about the data from that <date>
                record_IO_<date>.pbr   # protobuf data for faster model training
        hydrophone/
            raw/
                ...
            tabular/
                <date>.csv             # actual data file in tabular format
                meta_info_<date>.txt   # file containing meta info about the data from that <date>
                record_IO_<date>.pbr   # protobuf data for faster model training
        point_sensors/
            raw.csv
            tabular/
                <date>.csv             # actual data file in tabular format
    model/                             # models from training jobs of the organizing team     
```

Now let's see how much data volume we are handling for the tabular data:

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://gdsc4-eu/data/ek60/tabular/20 | grep "Total"

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://gdsc4-eu/data/adcp/tabular/20 | grep "Total"

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://gdsc4-eu/data/hydrophone/tabular/20 | grep "Total"

In [ ]:
!aws s3 ls --summarize --human-readable --recursive s3://gdsc4-eu/data/point_sensors/tabular/20 | grep "Total"

Note: the addition of the "20" to the end of the path is used to only select the actual data files and leave out the recordIO and meta_info files.

From here on we will switch to the use of Python for interaction with the S3 bucket. To make this a little easier we have two basic methods to list files under a specific prefix and to download a file from S3 to the disk of our Sagemaker Notebook / Sagemaker Studio instance:

In [ ]:
def get_file_list_from_s3(bucket_name, prefix):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    file_list = list(bucket.objects.filter(Prefix=prefix))
    return [element.key for element in file_list]


def download_file(bucket_name, s3_file_path, local_output_path):
    s3 = boto3.resource('s3')
    file_name = s3_file_path.split('/')[-1]
    try:
        s3.Bucket(bucket_name).download_file(
            str(s3_file_path), str(Path(local_output_path, file_name))
        )
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise  

#### _Echosounder - EK60_

A scientific echosounder is a sonar that transmits directional sound pulses through the water column.

The reflection (backscattering) from marine organisms such as fish and zooplankton are recorded providing information on vertical distribution and density, and in some cases allows for species identification.

Echosounder data are stored in vendor specific file formats, e.g. Kongsberg Maritime .raw files which can be read with open source Python libraries, e.g. [PyEchoLab](https://github.com/CI-CMG/pyEcholab). The files contain, in addition to other information about the sensor and environment, time, range, and volume backscattering strength. Echosounder data are visualized as echograms (time, range, volume backscattering strength).

We now will download a collection of small files containing meta information about the corresponding actual data files. The have the naming pattern *meta_info_\<date\>.txt* and can be found in the S3 bucket along with the data files in the folder */data/\<sensor_name\>/tabular/*. Based on those files we will answer the question of how many data points we have available and how they are distributed over time.
    
So let's get this data:

In [ ]:
ek60_prefix = "data/ek60/tabular"
ek60_meta_info_files = get_file_list_from_s3(bucket, f"{ek60_prefix}/meta")
local_ek60_file_path = Path(Path.cwd(), "data/ek60")

In [ ]:
# Downloading these files takes a short while - but not as long as the actual data files themselves
for meta_info_file in ek60_meta_info_files:
    download_file(bucket, meta_info_file, local_ek60_file_path)

What information is available within a "meta_info_*.txt" file?

In [ ]:
!cat data/ek60/meta_info_2013-09-30.txt

In [ ]:
local_ek60_meta_info_files = list(Path(Path.cwd(), "data/ek60").glob("meta_info_*.txt"))
local_ek60_meta_info_files = sorted(local_ek60_meta_info_files, key=lambda file_path: file_path.stem)

For easily extracting the needed information form the meta_info_*.txt files we use two small methods:

In [ ]:
def extract_number_of_records(meta_info_file_path):
    with open(Path(meta_info_file_path), "r") as f:
        text = f.read()
    return int(text.split('\n')[1].split('=')[1])

def extract_date(meta_info_file_path):
    return meta_info_file_path.stem.split('_')[2]

**How many EK60 echosounder data points do we have in total?**

In [ ]:
ek60_total_number_of_records = 0
for meta_info_file in local_ek60_meta_info_files:
    number_of_records = extract_number_of_records(meta_info_file)
    ek60_total_number_of_records += number_of_records
    
print(f"Total number of ek60 data points: {ek60_total_number_of_records}")

**How is the data distributed over time?**

In order to visualize the data points availability over time let's collect the number of records per day in a dataframe. We use two small method to extract the necessary infos from the meta_info files.

In [ ]:
# Get the infos from the meta_info_* files
ek60_dates = [datetime.strptime(extract_date(meta_info_file), "%Y-%m-%d") for meta_info_file in local_ek60_meta_info_files]
ek60_number_of_records = [extract_number_of_records(meta_info_file) for meta_info_file in local_ek60_meta_info_files]

# Put into a dataframe with a DatetimeIndex for easier date based querying
ek60_data_distribution = pd.DataFrame({"Records": ek60_number_of_records}, index=pd.DatetimeIndex(ek60_dates))
ek60_data_distribution.index.name = "Date"

In [ ]:
ek60_x_ticks = pd.date_range(start=min(ek60_data_distribution.index.date), end=max(ek60_data_distribution.index.date), freq='D')
plt.figure(figsize=(20,5))
plt.bar(x=ek60_data_distribution.index, height=ek60_data_distribution["Records"])
plt.xticks(ek60_x_ticks[0::20], [x_tick.strftime("%Y-%m-%d") for x_tick in ek60_x_ticks[0::20]], rotation=90)
plt.title("Number of EK60 Records per Day")
plt.ylabel("Number of Records")
plt.xlabel("Date")

From this plot we can see that for most days we have over 1400 data points. This aligns nicely with the fact that we expect to have one data point each minute and we therefore expect to have 60 * 24 = 1440 data points each day.
What we also can see is that we have gaps in our data. There is a big chunk of data missing from 2014-06 to 2014-10. Also we lack data for at least 2013-11 and 2015-07. The irregularities in the blocks also show that there are times when data came in kind of sparse i.e. we do have only a fraction of the 1440 data points per day - 2015-01 to 2015-03 is one example for this. 

Keep this overview in mind when selecting the data for model training. This becomes particularly interesting when you compare it to the data availability and data distribution of the other sensors.
> Exercise: create similar overview plots for the other sensors

<a id='section2.2'></a>
#### _Point Sensor Data_

By point sensors we denote a whole collection of simple sensors. Those simple sensors used at the LoVe Ocean Observatory include CTD (temperature, salinity/conductivity, density), Pressure, turbidity and chlorophyll. See table above for information on what these features mean.

These provide important information about the ocean environment.

For the point sensors we will use the raw version of the data for our analysis here. The tabular data will become handy down the line when we get to combining all the different data sources into a more comprehensive data set for model training across the various sensors. 

Let's download the raw data file for the point sensors. This already comes in CSV format

In [ ]:
point_sensors_prefix = "data/point_sensors"
local_point_sensors_file_path = Path(Path.cwd(), "data/point_sensors")
download_file(bucket, "data/point_sensors/raw.csv", local_point_sensors_file_path)

Again we load the data into a dataframe that has a [DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) and harmonize the column names:

In [ ]:
raw_point_sensors_data = pd.read_csv(
    Path(Path.cwd(), "data/point_sensors/raw.csv"), 
    index_col=0, 
    parse_dates=True
).rename(str.lower, axis='columns')
raw_point_sensors_data

In the table we can see that each sensor roughly has one measurement per hour but not all at the exact same time, with NaN values for the times where other sensors have measurements. Therefore we will bring them on the same sample frequency of one hour. Also, we limit the table to the years we are interested in and remove rows with only NaN values.

In [ ]:
point_sensors_data = raw_point_sensors_data.groupby(pd.Grouper(freq="H")).mean()["2013-01-01":"2016-01-01"]
point_sensors_data = point_sensors_data.dropna(axis=0, how='all')
point_sensors_data

This table now let's us answer the questions surrounding total number of data points and data distribution over time for the individual point sensors. To receive a result we can compare to the other sensors we will aggregate also the point sensors to a daily level. As an example we will do this for temperature. 

> Exercise: plot the remaining sensors and check their data availability over time

In [ ]:
minutes_per_hour = 60
temperatur_point_sensors_data_availability = pd.DataFrame(point_sensors_data["temp_c_at_depth"].groupby(pd.Grouper(freq="D")).count() * minutes_per_hour)
temperatur_point_sensors_data_availability

In [ ]:
temperature_point_sensors_data_availability_x_ticks = pd.date_range(
    start=min(temperatur_point_sensors_data_availability.index.date), 
    end=max(temperatur_point_sensors_data_availability.index.date), 
    freq='D'
)
plt.figure(figsize=(20,5))
plt.bar(x=temperatur_point_sensors_data_availability.index, height=temperatur_point_sensors_data_availability["temp_c_at_depth"])
plt.xticks(
    temperature_point_sensors_data_availability_x_ticks[0::20], 
    [x_tick.strftime("%Y-%m-%d") for x_tick in temperature_point_sensors_data_availability_x_ticks[0::20]], 
    rotation=90,
)
plt.title("Number of Temperature Records per Day")
plt.ylabel("Number of Records")
plt.xlabel("Date")

Here we see the same issues as with the echosounder data. We have a big gap where we lack data (2014-06 to 2014-10) plus multiple times periods with smaller gaps and where data is not available for the full day.

> Exercise: the meta_info files also contained information on #Dimensions - does this number stay constant for each data source? Should it stay constant and why would that be a good thing?

<a id='DataViz'></a>
## Data Visualization
One very important aspect of EDA is data visualization. Since we humans are visual creatures and process information based on what we see, deriving insights from the data we have in front of us becomes a lot easier if we find the right way to represent it in a visual form. So, now we want to go through some common ways to visualize our data. Depending on the source the approaches differ which is why we will go through the data source by source once again.

### Point sensors 
#### Value visualization
We will start with the simplest data source, the point sensors. Oncce again we will use temperature as an example, the other features can be visualized in a similar fashion.
> Exercise: visualize the other point sensors with different level of detail - drill down e.g. per month, per day
> Exercise: try out different averaging methods and window sizes to make patterns better visible

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(point_sensors_data["temp_c_at_depth"], label="raw")
plt.plot(point_sensors_data["temp_c_at_depth"].rolling(window=12).mean(), label="12h avg")
plt.title("Temperature over time")
plt.ylabel("Temperature in °C")
plt.xlabel("Time") 
plt.legend()

This plot provides us with several insights into our data.

For starters, we obviously have a big gap. For the temperature sensor there are about three months of data missing (2014-07 to 2014-10). A general pattern we can nonetheless detect is that we have the highest temperature points in the months of November and December and the lowest temperatures in and around April. Also, the magnitude of the temperature scale is a finding to note. Our temperature values only range from about 6°C to slightly about 9°C, so not that much variation.

> Exercise: verify dates of the minimal and maximal temperatures based on the data table

Based on this min and max values for the temperature we make the assumption that the seasonal temperature pattern at the site of the sensors (in the water) is somewhat shifted to what we are used to from ourly lives. E.g. for Europe we would expect to see the lowest temperatures in winter (like November to January/February) and the highest temperatures in summer (like June to Augest).

#### Feature Correlation
Here we show a heatmap and clustergrid across all our point sensors. These visualizations help us analyze correlated features and are particularly important if we want to remove redundant features. The heatmap computes a similarity score across each feature and colors like features using this score. The clustergrid is similar, however it presents feature correlations hierarchically.

**Note**: For the purposes of this notebook we do not remove any features but by gathering the findings from these plots one may choose to and can do so at this point. 

In [ ]:
plt.figure(figsize=(14,12))
cor = point_sensors_data.corr()
sns.heatmap(cor, annot=True, cmap=sns.diverging_palette(20, 220, n=5500))
plt.show()

In terms of correlation not too many direct things stick out. We see high correlation for "salinity" and "conductivity". This can be expected due to the scientific principles behind. The high correlation of the two temperature values is also nothing surprising. In general correlation plots should always be considered when investigating data with several dimensions. In our case it did not put forward anything surprising but since the correlations are present where you expect them this also works as a small data validation step for us.

Next we will go and have a look at one of the more sophisticated sensors we have - the Echosounder.

### EK60 - Echosounder data visualization

Before we visualize the Echosounder data we need to access it. So far we only looked at the meta_info_*.txt files. Let's get a few days of data downloaded for visualization. At the moment we do not know which ones are worth looking at but we aim to pick at a date where we have the full data of that date available. In February 2015 the chances seem to be pretty good from the plots we have seen so far, so we pick a date from that month - 2015-02-21.

In [ ]:
# convenience method to download data from a specific day - all our tabular data is stored in files on a per day basis
def download_daily_file(bucket, sensor_type, year, month, day, local_filepath):
    sensor_types = ["adcp", "ek60", "hydrophone", "point_sensors"]
    assert sensor_type in sensor_types, f"Please use one of the following for 'sensor_type' input arguments: {sensor_types}"
    date_of_interest = datetime(year, month, day)
    file_of_interest = f"data/{sensor_type}/tabular/{date_of_interest.strftime('%Y-%m-%d')}.csv"
    download_file(bucket, file_of_interest, Path(local_filepath)) 

We set the variables for the date that is of interest us

In [ ]:
year = 2015
month = 2
day = 21

In [ ]:
download_daily_file(bucket, "ek60", year, month, day, local_ek60_file_path)

We load the data and have a look at the data table

In [ ]:
ek60_data = pd.read_csv(
    Path(Path.cwd(), f"data/ek60/{year}-{month:02d}-{day:02d}.csv"),
    index_col=0,
    sep=',',
    parse_dates=True,
    header=0,
)
ek60_data["2015-02-21 02:00":"2015-02-21 04:00"]

Note that in the index of the dataframe we can see that we have one data point each minute. Each column provides the data at a certain distance away from the echosounder sensor, so meters above the sea floor.

Now we create the visualization for the echsounder

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(
    ek60_data.T,
    origin='lower',
    aspect='auto',
    cmap=plt.get_cmap('plasma'),
    interpolation=None,
    vmin=-90,
    vmax=-30
)

y_ticks = np.arange(0, ek60_data.shape[1], ek60_data.shape[1]//10).tolist()
m_per_datapoint = float(ek60_data.columns[1].split('_')[2].split('m')[0]) - float(ek60_data.columns[0].split('_')[2].split('m')[0])
y_ticks_labels = [int(yv * m_per_datapoint) for yv in y_ticks]
plt.yticks(y_ticks, y_ticks_labels)
plt.xticks(list(range(0, 1440))[::60], [f"{el:02d}:00" for el in range(0, 24)], rotation=90)

plt.colorbar(format="%2.f dB")
plt.xlabel("Time")
plt.ylabel("Meters")
plt.title("EK60 Echosounder - Volume Backscatter Strength")

This visualization of the echosounder data shows us the density in the water column above the sensor over time for one day. The broad yellow line on top depicts the ocean surface. Everything above it is no longer in the water, so the recordings there are not accurate and can not be used for our anomaly detection.

On this particular day three things are sticking out to the human eye and therefore could very well be anomalies we are looking for:
* The structure on the lower right (i.e. late in the evening on the sea floor)
* The higher density cloud in the top center (i.e. upper half of the water column around midday)
* The big amount of brighter sprinkles in the upper half (i.e. upper half of the water column throughout the entire day)

These structures are of interest to the LoVe Observatory reserach team. Not all of those have to be an anomaly but they are good candidates. What kind of possible anomaly those structures represent is not known to us and also not relevant for our task. We simply aim to find anomalies but do **not** need to classify the kind of anomaly that we are seeing.

Let's look at another echogram to check how it differs:

In [ ]:
year = 2015
month = 2
day = 28

In [ ]:
download_daily_file(bucket, "ek60", year, month, day, local_ek60_file_path)

In [ ]:
ek60_data = pd.read_csv(
    Path(Path.cwd(), f"data/ek60/{year}-{month:02d}-{day:02d}.csv"),
    index_col=0,
    sep=',',
    parse_dates=True,
    header=0,
)
ek60_data

In [ ]:
plt.figure(figsize=(25,10))
plt.imshow(
    ek60_data.T,
    origin='lower',
    aspect='auto',
    cmap=plt.get_cmap('plasma'),
    interpolation=None,
    vmin=-90,
    vmax=-30
)

y_ticks = np.arange(0, ek60_data.shape[1], ek60_data.shape[1]//10).tolist()
m_per_datapoint = float(ek60_data.columns[1].split('_')[2].split('m')[0]) - float(ek60_data.columns[0].split('_')[2].split('m')[0])
y_ticks_labels = [int(yv * m_per_datapoint) for yv in y_ticks]
plt.yticks(y_ticks, y_ticks_labels)
plt.xticks(list(range(0, 1440))[::60], [f"{el:02d}:00" for el in range(0, 24)], rotation=90)

plt.colorbar(format="%2.f dB")
plt.xlabel("Time")
plt.ylabel("Meters")
plt.title("EK60 Echosounder - Volume Backscatter Strength")

In this image we see a lot less activity. There are also structures which could be potential anomalies, mostly in the upper half throughout the day and especially in the late afternoon throughout the evening we see more purple parts. Also the ocean surface is a lot moer in motion compared to the day of data we looked at before. On the other hand we do have a much bigger part of the visualization wheere we see a deep blue area with almost no changes whatsoever.

The more data you look at like this, the better your gut feeling will become to do non-expert validation on your findings, so keep on exploring and checking out that data.

Now that we looked at the visualization for the echosounder there are quite a lot more things to look at. Hence, here are some exercises.

> Exercise: create visualizations of other days and see if some of them stick out compared to the rest - perform manual visual anomaly detection

> Exercise: create similar plots for the ADCP and Hydrophone data - they are avaialbe in CSV format as well and can be visualized along the same lines as we did with the echosounder

> Exercise: create hourly aggregates of the data and create plots for a full week or month

> Exercise: The yellow line at the top of the echosounder plot depicts the ocean surface. Find a way to more accurately detect the surface and remove the data values that lie beyond it.

> Advanced Exercise: The hydrophone is used to detect sounds in the ocean - communicating mamals are of particular interest. Check on the internet for mapping of particular mamals to frequencies that their sounds have. Given a frequency range, try to find a simple detection algorithm that focuses on events in this frequency range. 

For now this is the end of our EDA. We only performed this procedure on a small part of our data but the approaches used can be transfered to the other parts of our data.

Since big parts of the data engineering have already been performed with the conversion of the raw data to CSV format and the alignment of the sampling rate to one data point per minute we will now go directly to the model training. We will train a model based solely on the echosounder data. Combining the data sources and the engineering parts necessary for this will be part of the next tutorial.

***
<a id='Modelling'></a>
# Modelling

<a id='ModelInto'></a>
## Introduction

Random Cut Forest (RCF) Algorithm
Amazon SageMaker Random Cut Forest (RCF) is an unsupervised algorithm for detecting anomalousdata points within a data set. These are observations which diverge from otherwise well-structured or patterned data. Anomalies can manifest as unexpected spikes in time series data, breaks in periodicity, or unclassifiable data points. They are easy to describe in that, when viewed in a plot, they are often easily distinguishable from the "regular" data. Including these anomalies in a data set can drastically increase the complexity of a machine learning task since the "regular" data can often be described with a simple model. With each data point, RCF associates an anomaly score. Low score values indicate that the data point is considered "normal." High values indicate the presence of an anomaly in the data. The definitions of "low" and "high" depend on the application but common practice suggests that scores beyond three standard deviations from the mean score are considered anomalous. While there are many applications of anomaly detection algorithms to one-dimensional time series data such as traffic volume analysis or sound volume spike detection, RCF is designed to work with arbitrary dimensional input. 

Amazon SageMaker RCF scales well with respect to number of features, data set size, and number of instances.

In the next section we aim to learn how
* to create an AWS SageMaker training job on a data set to produce an RCF model,
* to use the RCF model to perform inference with an Amazon SageMaker endpoint.
* to use the model to identify and visualize anomalous timestamps.  

The following are ***not*** goals of this notebook:
* deeply understand the RCF model,
* understand how the Amazon SageMaker RCF algorithm works - this is answered [here](https://docs.aws.amazon.com/sagemaker/latest/dg/rcf_how-it-works.html)

However, the main idea behind the RCF algorithm is to create a forest of trees where each tree is obtained using a partition of a sample of the training data. For example, a random sample of the input data is first determined. The random sample is then partitioned according to the number of trees in the forest. Each tree is given such a partition and organizes that subset of points into a k-d tree. The anomaly score assigned to a data point by the tree is defined as the expected change in complexity of the tree as a result adding that point to the tree; which, in approximation, is inversely proportional to the resulting depth of the point in the tree. The random cut forest assigns an anomaly score by computing the average score from each constituent tree and scaling the result with respect to the sample size.

If you would like to know more please check out the [SageMaker RCF Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html) and a nice [blog article](https://aws.amazon.com/blogs/machine-learning/use-the-built-in-amazon-sagemaker-random-cut-forest-algorithm-for-anomaly-detection/) explaining some further details.

<a id='Training'></a>
## Training

Next, we configure a SageMaker training job to train the Random Cut Forest (RCF) algorithm on the echosounder data

### Hyperparameters

Particular to a SageMaker RCF training job are the following hyperparameters:

* **`num_samples_per_tree`** - the number randomly sampled data points sent to each tree. As a general rule, `1/num_samples_per_tree` should approximate the the estimated ratio of anomalies to normal points in the dataset.
* **`num_trees`** - the number of trees to create in the forest. Each tree learns a separate model from different samples of data. The full forest model uses the mean predicted anomaly score from each constituent tree.
* **`feature_dim`** - the dimension of each data point.

In addition to these RCF model hyperparameters, we provide additional parameters defining things like the EC2 instance type on which training will run, the S3 bucket containing the data, and the AWS access role. Note that,

* Recommended instance type: `ml.m4`, `ml.c4`, or `ml.c5`
* The RCF algorithm does not take advantage of GPU hardware.

You may have noticed that there are not many hyperparameters to tune. Therefore, this implementation is a good way to get started with tree based models. In addition to the limited number of hyperparameters, a lot of the work is done by the sampling method.

The RCF implementation uses Reservoir sampling, which is an algorithm for efficiently drawing random samples from a dataset where the elements in the dataset can only be observed one at a time or in batches. In fact, reservoir sampling works even when is not known a priori. 

Let's get started!

### Input/Output Interface for the RCF Algorithm

- Amazon SageMaker Random Cut Forest supports the train and test data channels. Channels are a named input source that AWS training algorithms can consume. More details [here](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_Channel.html).
- The optional test channel is used to compute accuracy, precision, recall, and F1-score metrics on labeled data. 
- Train and test data content types can be either application/x-recordio-protobuf or text/csv formats. For the test data, when using text/csv format, the content must be specified as text/csv;label_size=1 where the first column of each row represents the anomaly label: "1" for an anomalous data point and "0" for a normal data point. 
- You can use either File mode or Pipe mode to train RCF models on data that is formatted as recordIO-wrapped-protobuf or as CSV. The benefits of pipe mode are described in this [blog](https://aws.amazon.com/blogs/machine-learning/using-pipe-input-mode-for-amazon-sagemaker-algorithms/). The difference in how they work is the following:
    * File mode will copy the data from S3 to the hard drive of the training job instance for use in the algorithm training
    * Pipe mode will stream the data directly from S3 to the algorithm for training.

When possible we encourage you to use pipe mode in order to reduce training times and by that save costs. However, start with small amounts of data in file mode to develop your software and later on progress to pipe mode when your approach reached a certain level of maturity and you want to scale.

Here we will only train a model on a small subset of the data (one month instead of the roughly two years) in order to show you the way. To improve the model you would of course train it on a wider range of the data.

We start by checking how many datapoints we will use for the model training. We will achieve this by again making use of the meta_info_*.txt files.

In [ ]:
total_number_of_records = 0
for meta_info_file in list(Path('data/ek60').glob("meta_info_2015-03*.txt")):
    number_of_records = extract_number_of_records(meta_info_file)
    total_number_of_records += number_of_records
    
total_number_of_records

Next we need to create the proper python object containing the information about the training data to feed to the fit method of the model. For this the RecordSet class is used

In [ ]:
prefix = "data/ek60/tabular"

train_data = RecordSet(
    s3_data=f's3://{bucket}/{prefix}/recordIO_2015-03',
    num_records=total_number_of_records,
    feature_dim=1600,
    s3_data_type='S3Prefix',
    channel='train'
)
train_data

We now can create our model based on the AWS SageMaker built-in algorithm Randon Cut Forrest. Compared to other model or estimator objects that you create when doing machine learning with e.g. scikit learn one major difference here with SageMaker is the fact that you also provide the "instance_type" as an input parameter. This value determines the kind of compute instance that is used during the model training. I.e. the model is not trained on the instance we are working on right now - for our work so far we used a quite small instance and therefore saved a lot of costs and resources.

In [ ]:
participant_bucket = "gdsc4-tutorials" # set this to the name of your teams S3 bucket

rcf = sagemaker.RandomCutForest(
    role=role,
    instance_count=1,
    instance_type="ml.c5.xlarge",
    data_location=f"s3://{bucket}/{prefix}/",
    output_path=f"s3://{participant_bucket}/models/output",
    num_samples_per_tree=512,
    num_trees=50,
)

The next step in RCF is to construct a random cut forest using the random sample of data. First, the sample is partitioned into a number of equal-sized partitions equal to the number of trees in the forest. Then, each partition is sent to an individual tree. The tree recursively organizes its partition into a binarytree by partitioning the data domain into bounding boxes.
The RCF algorithm organizes these data in a tree by first computing a bounding box of the data, selecting a random dimension (giving more weight to dimensions with higher "variance"), and then randomly determining the position of a hyperplane "cut" through that dimension. The two resulting subspaces define their own sub tree. In this example, the cut happens to separate a lone point from the remainder of the sample. 

In [ ]:
rcf.fit(train_data)

#### If you see the message

> `===== Completed - Training job completed =====`

at the bottom of the output logs then that means training successfully completed and the output RCF model was stored in the specified output path. You can also view information about and the status of a training job using the AWS SageMaker console. Just click on the "Jobs" tab and select training job matching the training job name.

<a id='Inference'></a>
## Inference
When performing inference using a trained RCF model the final anomaly score is reported as the average across scores reported by each tree. Note that it is often the case that the new data point does not already reside in the tree. To determine the score associated with the new point the data point is inserted into the given tree and the tree is efficiently (and temporarily) reassembled in a manner equivalent to the training process described above. That is, the resulting tree is as if the input data point were a member of the sample used to construct the tree in the first place. The reported score is inversely proportional to the depth of the input point within the tree.

Here as well we can specify the "instance_type" when we deploy our model as an endpoint. For inference we usually get by with a smaller instance but we need to ensure to have enough memory for the model to be loaded, the m5 series is quite good for this. For this quick presentation we will however use a c5 instance to be on the save side.

In [ ]:
rcf_inference = rcf.deploy(initial_instance_count=1, instance_type="ml.c5.2xlarge")

Congratulations! You now have a functioning SageMaker RCF inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console and selecting the endpoint matching the endpoint name, below: 

In [ ]:
print(f"Endpoint name: {rcf_inference.endpoint}")

In [ ]:
# Implements methods for serializing data for an inference endpoint.  
# Serializing is the process of transforming data so that it can be stored, transmitted, or reconstructed later.
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

rcf_inference.serializer = CSVSerializer()
rcf_inference.deserializer = JSONDeserializer()

To check how our model performs we will make some predictions and get anomaly scores from the endpoint. To do that we will download a few days of data. This make take a short while - so keep that in mind when executing the cell. 

Also, when running inference on the full data set you might not want to download everything to the machine but check other ways for this, like [batch transform jobs](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-model-deployment.html#ex1-batch-transform). 

In [ ]:
csv_files = get_file_list_from_s3(bucket, f"{prefix}/2015-03")
for csv_file in csv_files:
    download_file(bucket, csv_file, Path('data/inference'))   

We will predict the anomaly scores for the data we downloaded and put the results in a dataframe containing the timestamp of the datapoint as well as its anomaly score.

This cell can take quite some time to execute - depending on how much data are using/downloaded.

In [ ]:
#local_csv_files = list(Path("data/inference").glob("2015-03*.csv"))
local_csv_files = [Path("data/ek60/2015-02-21.csv"), Path("data/ek60/2015-02-28.csv")] # use this smaller set of files in order to check against the findings of the data visualization

results = {
    "timestamps": list([]),
    "anomaly_score": list([]),
}
counter = 1
chunk_size = 100 # amount of data points send to the model in one batch for inference

for csv_file in local_csv_files:
    data_frame = pd.read_csv(csv_file, index_col=0, sep=',', parse_dates=True, header=0)
    data = data_frame.to_numpy()
    print(f"Processing file {counter} out of {len(local_csv_files)}")
    for chunk in [data[i:i + chunk_size, :] for i in range(0, len(data), chunk_size)]:
        result = rcf_inference.predict(
            chunk,
            initial_args={"ContentType": "text/csv", "Accept": "application/json"}
        )
        results["anomaly_score"].extend([entry["score"] for entry in result["scores"]])
    results["timestamps"].extend(data_frame.index)
    counter += 1

print("Predictions done")

In [ ]:
len(results['anomaly_score'])

In [ ]:
results_df = pd.DataFrame(
        {"anomaly_score": results["anomaly_score"]}, 
        index=pd.DatetimeIndex(results["timestamps"]),
)
results_df

> Exerscise: when doing predictions for other parts of the data - what can be done as a qualitative check to ensure the endpoint is working correctly?

From this result table we will now determine the anomaly candidates our model provided us. This is done by setting a proper threshold above which we deem a data point with that score an anomaly. A common choice for this threshold is 3 \sigma, i.e. 3 times the standard deviation

In [ ]:
score_mean = results_df.mean()
score_std = results_df.std()
anomaly_score_threshold = score_mean + 3 * score_std
anomalies = results_df[results_df["anomaly_score"].values > anomaly_score_threshold.values]
anomalies

<a id='Submission'></a>
## Submitting results

The timestamps of these anomaly candidates now provide us the info we want to submit for validation by the LoVe Observatory and also to later get a good position on the leader board of the challenge:

In [ ]:
submission = anomalies.index.strftime('%Y-%m-%d %H:%M').tolist()
submission

To submit your results we set up an API to interact with that stores your results and also provides them to our website as well as the annotation system of the research team. We will no go through the few lines of code that you need to submit your results as well as check what your latest submission was and retrieve it again. (Note: the client implementation of the Submissions API provides some further functionality but we will not go into those details here).

For the Submission API to work and identify your team you need to provide the "Token" from the [profile page](https://gdsc.ce.capgemini.com/profile/) of the [GDSC website](https://gdsc.ce.capgemini.com/). 
There are multiple ways to provide this token to the submission client, we will use the one we deem most secure: create a file within your project and set the environment variable "TOKEN_PATH" according to the location of that file. Within the file there is just the token copied, nothing else. Within a notebook this can be achieved as follows:

In [ ]:
%set_env TOKEN_PATH=/root/access_token
!echo "Token Path environment variable: $TOKEN_PATH"

Now we import the Client class, create an instance of it and provide our list of timestamps and check the upload summary

In [ ]:
import gdsc4

submission_client = gdsc4.Client()

In [ ]:
upload_summary = submission_client.put_submission(submission)
upload_summary

To have another check that the upload was successfull we will ask the API to provide us the information on our last submission:

In [ ]:
last_submissions_metadata = submission_client.get_last_submission_metadata()
last_submissions_metadata

In [ ]:
last_submission_data = submission_client.get_last_submission()
last_submission_data

Congratulations, you just made your first submission for the Global Data Science Challenge!

<a id='Validation'></a>
## Human Validity Check

But besides the validation via the researchers and the score on the leaderboard we also want to cross check the results of our model. This can be done for example by visualizing the anomaly score on top of the data visualization we already created and check whether the timestamps our model deems to be anomalous match what our trained human anomaly detection model (our brain) would point as anomalies 

In [ ]:
ek60_file = Path("data/ek60/2015-02-21.csv") 
ek60_inference_data = pd.read_csv(
    ek60_file,
    index_col=0,
    sep=',',
    parse_dates=True,
    header=0,
)
ek60_inference_range = [float(column_name.split("_")[2].split("m")[0]) for column_name in ek60_inference_data.columns]

In [ ]:
fig, ax1 = plt.subplots(figsize=(25,10))
ax2 = ax1.twinx()

ax1.pcolormesh(
    pd.to_datetime(ek60_inference_data.index),
    ek60_inference_range,
    ek60_inference_data.to_numpy().T,
    cmap=plt.get_cmap('plasma'),
    vmin=-90,
    vmax=-30
)

ax1.set_xlabel("Time")
ax1.set_ylabel("Meters")

ax2.plot(
    results_df["2015-02-21 00:00": "2015-02-21 23:59"].index,
    results_df["2015-02-21 00:00": "2015-02-21 23:59"]["anomaly_score"],
    color="whitesmoke",
    linewidth=0.2,
)
ax2.scatter(
    anomalies["2015-02-21 00:00": "2015-02-21 23:59"].index, 
    anomalies["2015-02-21 00:00": "2015-02-21 23:59"]["anomaly_score"], 
    color="black"
)
ax2.set_ylabel("Anomaly Score")

fig.suptitle("EK60 Echosounder - Anomaly Candidates in Echogram")


In this plot we added the anomaly scores over time on top of the echogram. The thin grey time series indicates the anomaly score provided by the RCF model. Anomaly scores with black dots indicate an anomaly score above our threshold. Those are of particular interest and we want to check if at this time we also see an interesting structure in the echogram. Also we want to validate if the anomaly score values (their tendency to be lower or higher) match our human perception of "things being out of the ordinary in the echogram".

The good news is, times with high anomaly score show around stuctures that indicate special activities in the water. Also the inverse is true for the majority of the time. For low values in the grey time series we mostly see deep blue colors instead of purple sparkles. 

> Exercise: Create this visualization also for bigger time ranges and extend to weekly or monthly scale

We see that with this hyperparameter selection and anomaly cutoff threshold we receive reasonable results, so a good starting point for further enhancements and tuning. In general, the number of trees, sample size, and anomaly score cutoff are all parameters that a data scientist may need experiment with in order to achieve desired results. The use of a labeled test dataset allows the used to obtain common accuracy metrics for anomaly detection algorithms. For more information about Amazon SageMaker Random Cut Forest see the [AWS Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/randomcutforest.html).

Before we finish up we stop our endpoint:

In [ ]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)

***

<a id='Epilogue'></a>
# Wrap Up

Besides the data we looked at in this tutorial session, where we focused mainly on the EK60 echosounder sensor, for the challenge we have data from more sensors available to accomplish our task. So, try going through the steps we outlined here also with the other sensors. The notebooks doing so can also be shared with the other teams, e.g. via our MS Teams or a public repository within AWS. First, this opens up a feedback channel for your work and also it puts you in a good position for the #Community Prize.

Here also some further material to look into for the next steps, prepare for next weeks tutorial session:
* Using Tensorflow with AWS Sagemaker - [Example notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_script_mode_training_and_serving/tensorflow_script_mode_training_and_serving.ipynb)
* Using autoencoder models for anomaly detection - [Keras Introduction Article](https://keras.io/examples/timeseries/timeseries_anomaly_detection/)

## Recap
We used Amazon SageMaker Random Cut Forest to detect anomalous datapoints in the Echosounder dataset. In these data the anomalies were scored by averaging the scores across the data, reported by its constituent trees. However, the RCF algorithm is also capable of detecting when, for example, data breaks periodicity or uncharacteristically changes global behavior.

- This is known as shingling, and is a potential improvement when working with periodic data with known period.  [Read more about it here](https://aws.amazon.com/blogs/machine-learning/use-the-built-in-amazon-sagemaker-random-cut-forest-algorithm-for-anomaly-detection/) 
- Also, there are techniques for scoring and benchmarking RCF models that we have not covered here.  [Find them here](https://aws.amazon.com/blogs/machine-learning/use-the-built-in-amazon-sagemaker-random-cut-forest-algorithm-for-anomaly-detection/)
